In [1]:
import numpy as np
import os
import pandas as pd


"""
Testing Setup of parameters and algorithm based on smaller sequence
"""

'\nTesting Setup of parameters and algorithm based on smaller sequence\n'

In [2]:
seq = np.array([4,5,6,2,2,3,1,3,4,2,6,6,5,1,1,2,1,3,3,4,1,3,4,5,6,2,1,2,3,4])

In [3]:
seq = np.array([4,5,6,2,2,3,1,3,4,2,6,6,5,1,1,2,1,3,3,4,1,3,4,5,6,2,1,2,3,4])

seq = seq - 1

In [4]:
print(seq)

[3 4 5 1 1 2 0 2 3 1 5 5 4 0 0 1 0 2 2 3 0 2 3 4 5 1 0 1 2 3]


In [56]:
def forward(V, a, b, initial_distribution):
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]
 
    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            # Matrix Computation Steps
            #                  ((1x2) . (1x2))      *     (1)
            #                        (1)            *     (1)
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
 
    return alpha
 



def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))
 
    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
 
    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
 
    return beta
 
 

In [58]:
def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0]
    T = len(V)
 
    for n in range(n_iter):
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)
 
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 2):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = np.log(numerator) / np.log(denominator)
 
        gamma = np.sum(xi, axis=1)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
 
        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)
 
        b = np.divide(b, denominator.reshape((-1, 1)))
 
    return {"a":a, "b":b}

In [59]:
states = [1,2]

In [60]:
pi = np.array([0.5, 0.5])

In [61]:
A = np.array([[0.95, 0.05], [0.05, 0.95]])

In [62]:
A.shape

(2, 2)

In [63]:
#Fair dye have equal probability for all 6 observations
#Loaded dye have P(6|L) = 1/2 while remaining 5 observations have even probability

B = np.array([[1/6,1/6,1/6,1/6,1/6,1/6,],
              [0.1,0.1,0.1,0.1,0.1,0.5]])

In [64]:
B.shape

(2, 6)

In [65]:
print(A.shape)
print(B.shape)
print(seq.shape)
print(pi.shape)

(2, 2)
(2, 6)
(30,)
(2,)


In [67]:
print(baum_welch(seq,A,B,pi, n_iter=1000))

{'a': array([[0.5, 0.5],
       [0.5, 0.5]]), 'b': array([[0.21428571, 0.21428571, 0.17857143, 0.14285714, 0.10714286,
        0.14285714],
       [0.21428571, 0.21428571, 0.17857143, 0.14285714, 0.10714286,
        0.14285714]])}


In [41]:
os.chdir(r"/content/drive/My Drive/STA-ML/HW10")

path = os.getcwd()
print(path)

/content/drive/My Drive/STA-ML/HW10


In [42]:
data = pd.read_csv('hmm_pb2.csv',header=None)

In [43]:
X = np.squeeze(data.values)

In [44]:
V = X-1
print(V)

[5 3 1 ... 0 2 2]


In [45]:
print(baum_welch(V,A,B,pi))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in multiply
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in multiply
  del sys.path[0]


{'a': array([[nan, nan],
       [nan, nan]]), 'b': array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]])}


In [54]:

V

array([5, 3, 1, ..., 0, 2, 2])